## Evolutionary origin

Macaca - Mouse - Human

In [12]:
import os,re,glob
import pandas as pd
import numpy as np
from collections import Counter
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO

users_dir = "/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction"
GENOMEDIR = "/genomics/users/marta/genomes"
species=["mouse","macaca","opossum","platypus","chicken"]

## annotation file
annotation="/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/human/newReference_Resconstructed/gencode.v38.gffcompare.TestisLiverBrain.annotation.sorted.1transcript.sorted.NOchr.gtf"
transcript_gene=pd.read_csv("/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/human/newReference_Resconstructed/1transcript_1gene.reconstructed.csv")

## evoDir
evoDir=os.path.join(users_dir,"EvolutionaryOrigin_OrthoFinder")

def translate_dna_to_protein(dna_seq):
    return str(Seq(dna_seq).translate())

# Function to save DF in fasta format
def create_seqrecord_PROT(row):
    return SeqRecord(Seq(row['protein']), id=row['header'][:-4], description="")
def create_seqrecord_DNA(row):
    return SeqRecord(Seq(row['seq']), id=row['header'], description="")



In [ ]:
## get repre sequences - both DNA and PROTEINS
for s in species:
    print(s)
    riboseq_dir = os.path.join(users_dir,s,"RiboSeq/RiboQC_RiboNovel")

    ## get all candidates
    print("Reading all the candidate ORF sequences")
    df = pd.read_csv(os.path.join(riboseq_dir,"Annotation/candidateORF.fa"), header=None, sep="\t")
    ## convert to df
    candidates_fasta = pd.DataFrame({'header':df[0].iloc[::2].values, 'seq':df[0].iloc[1::2].values})
    candidates_fasta['header'] = candidates_fasta['header'].str[1:]
    candidates_fasta = candidates_fasta[~candidates_fasta['header'].str.contains("iORF")]

    all_repre_candidates = pd.DataFrame()
    for sample in os.listdir(os.path.join(riboseq_dir,"RibORF")):
        if sample.endswith("r1"):
            print(sample)

            repre = pd.read_csv(os.path.join(riboseq_dir,"RibORF",sample,"repre.valid.pred.pvalue.parameters.txt"), sep="\t")
            repre_fasta = candidates_fasta[candidates_fasta['header'].isin(repre.orfID.values.tolist())]
            repre_fasta['header'] = s + "_" +candidates_fasta['header']

            repre_fasta['protein'] = repre_fasta['seq'].apply(translate_dna_to_protein)
            all_repre_candidates = pd.concat([all_repre_candidates, repre_fasta])

            ## translate to protein
            print("Translating to protein")
            out_proteins = os.path.join(riboseq_dir,"RibORF",sample,"repre.valid.pred.pvalue.parameters.PROTEIN.fa")
            seq_records_proteins = repre_fasta.apply(create_seqrecord_PROT, axis=1).tolist()
            SeqIO.write(seq_records_proteins, out_proteins, "fasta")

            ## keep the dna sequences as well
            print("Saving the original DNA ORF")
            out_dna = os.path.join(riboseq_dir,"RibORF",sample,"repre.valid.pred.pvalue.parameters.DNA.fa")
            seq_records_dna = repre_fasta.apply(create_seqrecord_DNA, axis=1).tolist()
            SeqIO.write(seq_records_dna, out_dna, "fasta")

            ## get only the nonredundant
            print("Getting the non-redundant")
            all_repre_candidates.drop_duplicates(inplace=True)
            out_nonredundant_proteins = os.path.join(riboseq_dir,"RibORF")+"/"+s+"_repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.fa"
            seq_records_proteins = all_repre_candidates.apply(create_seqrecord_PROT, axis=1).tolist()
            SeqIO.write(seq_records_proteins, out_nonredundant_proteins, "fasta")

## Orthofinder

Against all expressed in mouse, macaca, platypus, opossum in any tissue (non-redunant set)

#### TESTIS-EXPRESSED

In [4]:
## Testis-expressed ORFs human
TestisExpressed_translatedINtestis = pd.read_csv("/projects_eg/projects/marta/TestisRestricted_Microproteins_TSA/with_TranscriptomeReconstruction/Q1_TestisORFs/human/ribORF_humanTestis_in1.csv")
TestisExpressed_translatedINtestis

def create_seqrecord_PROThuman(row):
    return SeqRecord(Seq(row['ORFpep']), id=row['orfID'], description="")

out_nonredundant_proteins = "/projects_eg/projects/marta/TestisRestricted_Microproteins_TSA/with_TranscriptomeReconstruction/Q1_TestisORFs/human/ribORF_humanTestis_in1.fa"
seq_records_proteins = TestisExpressed_translatedINtestis.apply(create_seqrecord_PROThuman, axis=1).tolist()
SeqIO.write(seq_records_proteins, out_nonredundant_proteins, "fasta")

15040

Creating database of human (expressed in testis)

In [12]:
%%bash -s "$evoDir" "$users_dir"

# mkdir $1/MMseqsDB

module load Miniconda3/4.9.2
source /soft/system/software/Miniconda3/4.9.2/bin/activate MMseqs2

mmseqs createdb /projects_eg/projects/marta/TestisRestricted_Microproteins_TSA/with_TranscriptomeReconstruction/Q1_TestisORFs/human/ribORF_humanTestis_in1.fa $1/MMseqsDB/ribORF_humanTestis_in1_DB


createdb /projects_eg/projects/marta/TestisRestricted_Microproteins_TSA/with_TranscriptomeReconstruction/Q1_TestisORFs/human/ribORF_humanTestis_in1.fa /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/EvolutionaryOrigin/MMseqsDB/ribORF_humanTestis_in1_DB 

MMseqs Version:       	14.7e284
Database type         	0
Shuffle input database	true
Createdb mode         	0
Write lookup file     	1
Offset of numeric ids 	0
Compressed            	0
Verbosity             	3

Converting sequences
[=
Time for merging to ribORF_humanTestis_in1_DB_h: 0h 0m 0s 518ms
Time for merging to ribORF_humanTestis_in1_DB: 0h 0m 0s 708ms
Database type: Aminoacid
Time for processing: 0h 0m 4s 150ms


Creating a joint database with all the specie, everything translated, no matter the tissue

In [24]:
%%bash -s "$evoDir" "$users_dir"

module load Miniconda3/4.9.2
source /soft/system/software/Miniconda3/4.9.2/bin/activate MMseqs2

# species="platypus mouse macaca chicken"
# for s in $species; do
#     echo $s

#     ### create DB
#     mmseqs createdb $2/$s/RiboSeq/RiboQC_RiboNovel/RibORF/repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.fa $1/MMseqsDB/${s}_noredunantProts_DB

# done

mmseqs createdb $2/*/RiboSeq/RiboQC_RiboNovel*/RibORF/*_repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.fa $1/MMseqsDB/speciesDB



/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/EvolutionaryOrigin/MMseqsDB/speciesDB exists and will be overwritten
createdb /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/chicken/RiboSeq/RiboQC_RiboNovel/RibORF/chicken_repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.fa /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/macaca/RiboSeq/RiboQC_RiboNovel/RibORF/macaca_repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.fa /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/mouse/RiboSeq/RiboQC_RiboNovel/RibORF/mouse_repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.fa /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/opossum/RiboSeq/RiboQC_RiboNovel_RiboDetector/RibORF/opossum_repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.fa /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/platypus/

Creating a joint database with all the specie AND human for clustering, everything translated for the species, no matter the tissue, but only translated in 1 for testis

In [25]:
%%bash -s "$evoDir" "$users_dir"

module load Miniconda3/4.9.2
source /soft/system/software/Miniconda3/4.9.2/bin/activate MMseqs2

mmseqs createdb $2/*/RiboSeq/RiboQC_RiboNovel*/RibORF/*_repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.fa /projects_eg/projects/marta/TestisRestricted_Microproteins_TSA/with_TranscriptomeReconstruction/Q1_TestisORFs/human/ribORF_humanTestis_in1.fa $1/MMseqsDB/species_and_humanDB



/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/EvolutionaryOrigin/MMseqsDB/species_and_humanDB exists and will be overwritten
createdb /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/chicken/RiboSeq/RiboQC_RiboNovel/RibORF/chicken_repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.fa /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/macaca/RiboSeq/RiboQC_RiboNovel/RibORF/macaca_repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.fa /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/mouse/RiboSeq/RiboQC_RiboNovel/RibORF/mouse_repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.fa /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/opossum/RiboSeq/RiboQC_RiboNovel_RiboDetector/RibORF/opossum_repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.fa /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction

In [26]:
%%bash -s "$evoDir" "$users_dir"

module load Miniconda3/4.9.2
source /soft/system/software/Miniconda3/4.9.2/bin/activate MMseqs2
mkdir -p $1/Searching

## The alignment consists of two steps the prefilter and alignment. To run the search, type:
mmseqs search -a -s 6 $1/MMseqsDB/ribORF_humanTestis_in1_DB $1/MMseqsDB/speciesDB $1/Searching/resultDB $1/tmp

search -a -s 6 /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/EvolutionaryOrigin/MMseqsDB/ribORF_humanTestis_in1_DB /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/EvolutionaryOrigin/MMseqsDB/speciesDB /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/EvolutionaryOrigin/Searching/resultDB /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/EvolutionaryOrigin/tmp 

MMseqs Version:                        	14.7e284
Substitution matrix                    	aa:blosum62.out,nucl:nucleotide.out
Add backtrace                          	true
Alignment mode                         	2
Alignment mode                         	0
Allow wrapped scoring                  	false
E-value threshold                      	0.001
Seq. id. threshold                     	0
Min alignment length                   	0
Seq. id. mode                          	0
Alternative alignments            

In [27]:
%%bash -s "$evoDir" "$users_dir"

module load Miniconda3/4.9.2
source /soft/system/software/Miniconda3/4.9.2/bin/activate MMseqs2

# Then, convert the result database into a BLAST tab formatted file (option -m 8 in legacy blast, -outfmt 6 in blast+):
# Format alignment output	query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits

mmseqs convertalis $1/MMseqsDB/ribORF_humanTestis_in1_DB $1/MMseqsDB/speciesDB $1/Searching/resultDB $1/Searching/resultDB.m8

convertalis /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/EvolutionaryOrigin/MMseqsDB/ribORF_humanTestis_in1_DB /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/EvolutionaryOrigin/MMseqsDB/speciesDB /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/EvolutionaryOrigin/Searching/resultDB /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/EvolutionaryOrigin/Searching/resultDB.m8 

MMseqs Version:        	14.7e284
Substitution matrix    	aa:blosum62.out,nucl:nucleotide.out
Alignment format       	0
Format alignment output	query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits
Translation table      	1
Gap open cost          	aa:11,nucl:5
Gap extension cost     	aa:1,nucl:2
Database output        	false
Preload mode           	0
Search type            	0
Threads                	24
Compressed             	0
Verbosity              	3

[======

In [28]:
%%bash -s "$evoDir" "$users_dir"

module load Miniconda3/4.9.2
source /soft/system/software/Miniconda3/4.9.2/bin/activate MMseqs2

mkdir -p $1/Clustering

## cluster all species and human
mmseqs cluster $1/MMseqsDB/species_and_humanDB $1/Clustering/species_and_human_cluDB $1/tmp
mmseqs createtsv $1/MMseqsDB/species_and_humanDB $1/MMseqsDB/species_and_humanDB $1/Clustering/species_and_human_cluDB $1/Clustering/species_and_human_cluDB.tsv

cluster /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/EvolutionaryOrigin/MMseqsDB/species_and_humanDB /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/EvolutionaryOrigin/Clustering/species_and_human_cluDB /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/EvolutionaryOrigin/tmp 

MMseqs Version:                     	14.7e284
Substitution matrix                 	aa:blosum62.out,nucl:nucleotide.out
Seed substitution matrix            	aa:VTML80.out,nucl:nucleotide.out
Sensitivity                         	4
k-mer length                        	0
k-score                             	seq:2147483647,prof:2147483647
Alphabet size                       	aa:21,nucl:5
Max sequence length                 	65535
Max results per query               	20
Split database                      	0
Split mode                          	2
Split memory limit                  	0
Coverage threshold                  

## Analysis

In [2]:
ribORF_in_2 = pd.read_csv("/projects_eg/projects/marta/TestisRestricted_Microproteins_TSA/with_TranscriptomeReconstruction/ribORF_humanTestis_in2.csv")
ribORF_in_2

,orfID,ORFseq,gene_id,gene_name,transcript_id,gene_type,orfType,length,length_aa,start_codon,ORFpep
0,ENST00000000412.8:12:-|10|2450:160:994|canonic...,ATGTTCCCTTTCTACAGCTGCTGGAGGACTGGACTGCTACTACTAC...,ENSG00000003056,M6PR,ENST00000000412,protein_coding,canonical,834,278,ATG,MFPFYSCWRTGLLLLLLAVAVRESWQTEEKTCDLVGEKGKESEKEL...
1,ENST00000001008.6:12:+|5|3715:171:1551|canonic...,ATGACAGCCGAGGAGATGAAGGCGACCGAGAGCGGGGCGCAGTCGG...,ENSG00000004478,FKBP4,ENST00000001008,protein_coding,canonical,1380,460,ATG,MTAEEMKATESGAQSAPLPMEGVDISPKQDEGVLKVIKREGTGTEM...
2,ENST00000001146.7:2:-|1|4556:29:1568|canonical...,ATGCTCTTTGAGGGCTTGGATCTGGTGTCGGCGCTGGCCACCCTCG...,ENSG00000003137,CYP26B1,ENST00000001146,protein_coding,canonical,1539,513,ATG,MLFEGLDLVSALATLAACLVSVTLLLAVSQQLWQLRWAATRDKSCK...
3,ENST00000002165.11:6:-|3|2385:87:1491|canonica...,ATGCGGCCCCAGGAGCTCCCCAGGCTCGCGTTCCCGTTGCTGCTGT...,ENSG00000001036,FUCA2,ENST00000002165,protein_coding,canonical,1404,468,ATG,MRPQELPRLAFPLLLLLLLLLPPPPCPAHSATRFDPTWESLDARQL...
4,ENST00000002596.6:4:-|24|7160:305:1229|canonic...,ATGGCCGCGCTGCTCCTGGGCGCGGTGCTGCTGGTGGCCCAGCCCC...,ENSG00000002587,HS3ST1,ENST00000002596,protein_coding,canonical,924,308,ATG,MAALLLGAVLLVAQPQLVPSRPAELGQQELLRKAGTLQDDVRDGVA...
...,...,...,...,...,...,...,...,...,...,...,...
12516,TCONS_00001992:8:+|9|1191:153:675|noncoding|ATG,ATGACCGGGTCAGCGGAAATTTCAGAGCCAGCCAGCAGCTCAGGTC...,XLOC_001676,XLOC_001676,TCONS_00001992,novel,noncoding,522,174,ATG,MTGSAEISEPASSSGPRGRGRNGTAPRRQPRGHSRATADSGTSLSA...
12517,TCONS_00002050:9:-|86|10655:1396:1420|noncodin...,CTGTGTAAGTGCATCAGAACATAA,XLOC_001809,XLOC_001809,TCONS_00002050,novel,noncoding,24,8,CTG,MCKCIRT*
12518,TCONS_00002051:9:-|9|437:109:145|noncoding|ATG,ATGGCTTTACCCATGGAGATTTTTGTTGCTGGCTAA,XLOC_001810,XLOC_001810,TCONS_00002051,novel,noncoding,36,12,ATG,MALPMEIFVAG*
12519,TCONS_00002066:9:+|15|2910:249:327|noncoding|ATG,ATGTGCTTAATAAATATACGCCGAATGACTGAACTGCAACCAAGGA...,XLOC_001758,XLOC_001758,TCONS_00002066,novel,noncoding,78,26,ATG,MCLINIRRMTELQPRTLYTCFTDLV*


In [3]:
## Analysis in Mouse
human_mouse = pd.read_csv(os.path.join(evoDir,"TestisExpressedHuman_vs_mouse.txt"), sep="\t", header=None)
human_mouse.columns = ["qseqid","sseqid","pident","length","mismatch","gapopen","qstart","qend","sstart","send","evalue","bitscore","qlen","slen","qframe","sframe","sstrand","qcovs"]

ribORF_in_2['mouse'] = np.where(ribORF_in_2['orfID'].isin(human_mouse.qseqid.values.tolist()), "conserved_mouse","no")
ribORF_in_2.groupby(['gene_type','mouse']).count()


orfID  ORFseq  gene_id  gene_name  \
gene_type            mouse                                                
lncRNA               conserved_mouse    110     110      110        110   
                     no                 394     394      394        394   
novel                no                  63      63       63         63   
processed_pseudogene conserved_mouse    109     109      109        109   
                     no                  14      14       14         14   
protein_coding       conserved_mouse  11533   11533    11533      11533   
                     no                 298     298      298        298   

                                      transcript_id  orfType  length  \
gene_type            mouse                                             
lncRNA               conserved_mouse            110      110     110   
                     no                         394      394     394   
novel                no                          63       63      63   
processed_pseudogene conserved_mouse            109      109     109   
                     no                          14       14      14   
protein_coding       conserved_mouse          11533    11533   11533   
                     no                         298      298     298   

                                      length_aa  start_codon  ORFpep  
gene_type            mouse                                            
lncRNA               conserved_mouse        110          110     110  
                     no                     394          394     394  
novel                no                      63           63      63  
processed_pseudogene conserved_mouse        109          109     109  
                     no                      14           14      14  
protein_coding       conserved_mouse      11533        11533   11533  
                     no                     298          298     298

In [4]:
## Analysis in Macaca
human_macaca = pd.read_csv(os.path.join(evoDir,"TestisExpressedHuman_vs_macaca.txt"), sep="\t", header=None)
human_macaca.columns = ["qseqid","sseqid","pident","length","mismatch","gapopen","qstart","qend","sstart","send","evalue","bitscore","qlen","slen","qframe","sframe","sstrand","qcovs"]

ribORF_in_2['macaca'] = np.where(ribORF_in_2['orfID'].isin(human_macaca.qseqid.values.tolist()), "conserved_macaca","no")
# ribORF_in_2_macaca.drop(columns=['mouse'], inplace=True)
ribORF_in_2.to_csv(os.path.join(evoDir,"ribORF_humanTestis_in2.evolution.csv"), index=None)
ribORF_in_2.groupby(['gene_type','macaca','mouse']).count()


orfID  ORFseq  gene_id  \
gene_type            macaca           mouse                                     
lncRNA               conserved_macaca conserved_mouse    109     109      109   
                                      no                  62      62       62   
                     no               conserved_mouse      1       1        1   
                                      no                 332     332      332   
novel                conserved_macaca no                   8       8        8   
                     no               no                  55      55       55   
processed_pseudogene conserved_macaca conserved_mouse    107     107      107   
                                      no                   7       7        7   
                     no               conserved_mouse      2       2        2   
                                      no                   7       7        7   
protein_coding       conserved_macaca conserved_mouse  11374   11374    11374   
                                      no                 269     269      269   
                     no               conserved_mouse    159     159      159   
                                      no                  29      29       29   

                                                       gene_name  \
gene_type            macaca           mouse                        
lncRNA               conserved_macaca conserved_mouse        109   
                                      no                      62   
                     no               conserved_mouse          1   
                                      no                     332   
novel                conserved_macaca no                       8   
                     no               no                      55   
processed_pseudogene conserved_macaca conserved_mouse        107   
                                      no                       7   
                     no               conserved_mouse          2   
                                      no                       7   
protein_coding       conserved_macaca conserved_mouse      11374   
                                      no                     269   
                     no               conserved_mouse        159   
                                      no                      29   

                                                       transcript_id  orfType  \
gene_type            macaca           mouse                                     
lncRNA               conserved_macaca conserved_mouse            109      109   
                                      no                          62       62   
                     no               conserved_mouse              1        1   
                                      no                         332      332   
novel                conserved_macaca no                           8        8   
                     no               no                          55       55   
processed_pseudogene conserved_macaca conserved_mouse            107      107   
                                      no                           7        7   
                     no               conserved_mouse              2        2   
                                      no                           7        7   
protein_coding       conserved_macaca conserved_mouse          11374    11374   
                                      no                         269      269   
                     no               conserved_mouse            159      159   
                                      no                          29       29   

                                                       length  length_aa  \
gene_type            macaca           mouse                                
lncRNA               conserved_macaca conserved_mouse     109        109   
                                      no                   62         62   
                     no               conserved_mouse       1       